# 1. Loading the dataset

## 1.1. Dataset metadata

The Features are:

    1. amount_tsh - Total static head (amount water available to waterpoint)
    2. date_recorded - The date the row was entered
    3. funder - Who funded the well
    4. gps_height - Altitude of the well
    5. installer - Organization that installed the well
    6. longitude - GPS coordinate
    7. latitude - GPS coordinate
    8. wpt_name - Name of the waterpoint if there is one
    9. num_private -
    10. basin - Geographic water basin
    11. subvillage - Geographic location
    12. region - Geographic location
    13. region_code - Geographic location (coded)
    14. district_code - Geographic location (coded)
    15. lga - Geographic location
    16. ward - Geographic location
    17. population - Population around the well
    18. public_meeting - True/False
    19. recorded_by - Group entering this row of data
    20. scheme_management - Who operates the waterpoint
    21. scheme_name - Who operates the waterpoint
    22. permit - If the waterpoint is permitted
    23. construction_year - Year the waterpoint was constructed
    24. extraction_type - The kind of extraction the waterpoint uses
    25. extraction_type_group - The kind of extraction the waterpoint uses
    26. extraction_type_class - The kind of extraction the waterpoint uses
    27. management - How the waterpoint is managed
    28. management_group - How the waterpoint is managed
    29. payment - What the water costs
    30. payment_type - What the water costs
    31. water_quality - The quality of the water
    32. quality_group - The quality of the water
    33. quantity - The quantity of water
    34. quantity_group - The quantity of water
    35. source - The source of the water
    36. source_type - The source of the water
    37. source_class - The source of the water
    38. waterpoint_type - The kind of waterpoint
    40. waterpoint_type_group - The kind of waterpoint

The target feature is `status_group` with the following categories:

    - functional - the waterpoint is operational and there are no repairs needed
    - functional needs repair - the waterpoint is operational, but needs repairs
    - non functional - the waterpoint is not operational

## 1.2. Import all necessary Libraries

In [22]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.impute import SimpleImputer

In [23]:
# setting some options
pd.set_option('display.max_columns', 60)
plt.style.use('ggplot')

## 1.3. Load the dataset

In [24]:
# the data I am using is in the data folder and are under the name 'training'
# the data is also separated into target and features already
df_features = pd.read_csv('/home/nick/Documents/Flatiron/Phase 3/Water_wells_Project/Data/training features.csv')
df_target = pd.read_csv('/home/nick/Documents/Flatiron/Phase 3/Water_wells_Project/Data/training targets.csv')

df_features.columns, df_target.columns

(Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
        'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
        'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
        'ward', 'population', 'public_meeting', 'recorded_by',
        'scheme_management', 'scheme_name', 'permit', 'construction_year',
        'extraction_type', 'extraction_type_group', 'extraction_type_class',
        'management', 'management_group', 'payment', 'payment_type',
        'water_quality', 'quality_group', 'quantity', 'quantity_group',
        'source', 'source_type', 'source_class', 'waterpoint_type',
        'waterpoint_type_group'],
       dtype='object'),
 Index(['id', 'status_group'], dtype='object'))

In [25]:
df_features.shape, df_target.shape

((59400, 40), (59400, 2))

In [26]:
# join the two to make one whole dataset
df = df_features.join(df_target, rsuffix='_target')

# drop id_target col
df.drop(columns='id_target', inplace=True)

df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [ ]:
# A function to drop columns
def drop_cols(dataframe, column=list(), inplace=False):
    pass

# 2. Explore the dataset structure

In [27]:
# check the info and shape of my data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [28]:
df.shape

(59400, 41)

I have 41 columns with 31 of them being of object dtype.

In [29]:
# Check the description
# this will only include columns of numeric datatypes
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [30]:
# now including the object dtype columns alone
df.describe(include='object')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,32259


## 2.1. Checking for Duplicates

In [31]:
# Checking for duplicates
df.duplicated().sum()

0

There are no duplicates in the data.

## 2.2. Checking for Missing values

In [32]:
missing_values = pd.DataFrame(df.isna().sum(), columns=['value'])
missing_values[missing_values['value'] != 0]

,value
funder,3635
installer,3655
subvillage,371
public_meeting,3334
scheme_management,3877
scheme_name,28166
permit,3056


The columns above are where the missing values are.

Scheme_name has approximately half of the records missing. 
This is very bad. Let me see what could be the issue.

In [33]:
# scheme is about who manages the waterpoint
# since columns with prefix scheme basically describe the same thing
# use both of them to see what may be the problem
df[['scheme_management', 'scheme_name']].head(20)

,scheme_management,scheme_name
0,VWC,Roman
1,Other,NaN
2,VWC,Nyumba ya mungu pipe scheme
3,VWC,NaN
4,NaN,NaN
5,VWC,Zingibali
6,VWC,NaN
7,NaN,NaN
8,VWC,NaN
9,NaN,NaN


It is a bit hard to deal with the missing data in scheme_name column. So I will just drop it.

In [34]:
df.drop(columns='scheme_name', inplace=True)

In [35]:
# let me see what the other columns with missing values look like
missing_col_list = ['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'permit']

# filter the dataframe to see the specified columns above
df[missing_col_list].head(20)

,funder,installer,subvillage,public_meeting,scheme_management,permit
0,Roman,Roman,Mnyusi B,True,VWC,False
1,Grumeti,GRUMETI,Nyamara,NaN,Other,True
2,Lottery Club,World vision,Majengo,True,VWC,True
3,Unicef,UNICEF,Mahakamani,True,VWC,True
4,Action In A,Artisan,Kyanyamisa,True,NaN,True
5,Mkinga Distric Coun,DWE,Moa/Mwereme,True,VWC,True
6,Dwsp,DWSP,Ishinabulandi,True,VWC,True
7,Rwssp,DWE,Nyawishi Center,True,NaN,True
8,Wateraid,Water Aid,Imalauduki,True,VWC,True
9,Isingiro Ho,Artisan,Mkonomre,True,NaN,True


All these columns are of object datatype. 

Therefore, the best method of dealing with the missing values will be filling them with the mode(most frequent) string.

In [36]:
# filling nan values
# instantiate the imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# fit the imputer
imputed = imputer.fit_transform(df[missing_col_list])

# create a dataframe of the result above
imputed_df = pd.DataFrame(imputed, columns=missing_col_list, index=df.index)

# check to see if it worked 
imputed_df.isna().sum()

funder               0
installer            0
subvillage           0
public_meeting       0
scheme_management    0
permit               0
dtype: int64

In [39]:
# drop the columns included in the transformation from the original dataframe
df = df.drop(columns=missing_col_list)

# then concatenate the new df with imputed_df
df = pd.concat([df, imputed_df], axis=1)
df.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,funder,installer,subvillage,public_meeting,scheme_management,permit
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,Roman,Roman,Mnyusi B,True,VWC,False
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,Grumeti,GRUMETI,Nyamara,True,Other,True
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,Lottery Club,World vision,Majengo,True,VWC,True
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,Unicef,UNICEF,Mahakamani,True,VWC,True
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,Action In A,Artisan,Kyanyamisa,True,VWC,True


In [132]:
df.funder.nunique()

1897

All nan values have been dealt with now.

## 2.3. Check for outliers in the numeric columns

This investigation will omit:
- any location data e.g latitude and longitude
- columns describing a time measure e.g year
- and any other column that doesn't make sense to look for outliers


In [62]:
# choose only numeric datatypes
numeric_df = df.select_dtypes(include='number').copy()

# drop latitude, longitude, construction_year, region_code, district_code
# include the id column for ease in merging back to original dataframe
numeric_df.drop(columns=['latitude', 'longitude', 'construction_year', 'region_code', 'district_code'], inplace=True)
numeric_df.head()

,id,amount_tsh,gps_height,num_private,population
0,69572,6000.0,1390,0,109
1,8776,0.0,1399,0,280
2,34310,25.0,686,0,250
3,67743,0.0,263,0,58
4,19728,0.0,0,0,0


In [63]:
# check the columns to see if any are categorical
numeric_df.nunique()

id             59400
amount_tsh        98
gps_height      2428
num_private       65
population      1049
dtype: int64

None of the numeric columns above are categorical. I can continue checking for outliers comfortably.

In [64]:
numeric_df.describe()

,id,amount_tsh,gps_height,num_private,population
count,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,0.474141,179.909983
std,21453.128371,2997.574558,693.116350,12.236230,471.482176
min,0.000000,0.000000,-90.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,0.000000,25.000000
75%,55656.500000,20.000000,1319.250000,0.000000,215.000000
max,74247.000000,350000.000000,2770.000000,1776.000000,30500.000000


There is a possibility of existence of outliers in all columns except `id`

### 2.3.1. amount_tsh

In [106]:
# checking the percentage of 0 values
zero_count_1 = numeric_df.amount_tsh.value_counts()[0]
zero_count_1 / len(numeric_df.amount_tsh)

0.7010286714817247

In [107]:
# checking the distribution of the top values
numeric_df.amount_tsh.sort_values(ascending=False).head(25)

45067    170000.0
9961     138000.0
9917     120000.0
57811    117000.0
44381    117000.0
45470    117000.0
5558     117000.0
10615    117000.0
54595    117000.0
37459    117000.0
6591     100000.0
3228     100000.0
38137    100000.0
21567     70000.0
53968     60000.0
50494     50000.0
47201     50000.0
51260     50000.0
12895     50000.0
39114     45000.0
26130     45000.0
11349     45000.0
3245      40000.0
44803     40000.0
59375     40000.0
Name: amount_tsh, dtype: float64

Approximately 70% of our data is recorded as 0 yet we have values reaching as far as 350,000.

I am going to drop all values > 199,999

In [108]:
numeric_df = numeric_df[numeric_df.amount_tsh < 200000]

### 2.3.2. gps_height

In [110]:
# checking the percentage of 0 values
zero_count_2 = numeric_df.gps_height.value_counts()[0]
zero_count_2 / len(numeric_df.gps_height)

0.3440914524302574

In [120]:
# checking the distribution of the top values
numeric_df.gps_height.sort_values(ascending=False).head(35000)

17374    2770
40822    2628
12905    2627
35453    2626
21388    2626
         ... 
13346     105
43141     105
21625     105
30418     105
41844     105
Name: gps_height, Length: 35000, dtype: int64

The distribution isn't that bad. So I'll just retain all values for now and decide later if there is need to drop them.

### 2.3.3. num_private

In [122]:
# checking the percentage of 0 values
zero_count_3 = numeric_df.num_private.value_counts()[0]
zero_count_3 / len(numeric_df.num_private)

0.9872552485815782

In [125]:
numeric_df.num_private.sort_values(ascending=False).head(60)

24657    1776
54982    1402
45425     755
727       698
17849     672
17828     668
40106     450
49528     300
21710     280
44892     240
2713      213
11703     180
24112     160
39453     150
39240     150
55710     150
20715     150
41155     150
40105     150
3383      141
8046      136
4244      131
36794     120
18363     120
58974     120
2535      120
54016     120
17337     120
32463     120
46932     111
57685     102
16840     102
38831     102
20319     102
11610     102
47156     102
19561     102
47079     102
5926      102
1752      102
42230     102
41480     102
49372     102
37895     102
8940      102
25175     102
20208     102
13089     102
17605     102
34803     102
32998      94
46661      93
6471       93
52370      93
7953       93
58587      93
48349      93
45084      93
46802      93
35530      93
Name: num_private, dtype: int64

Over 98% of the record here are 0. Yet only two are above 1000. This is a very great margin. 

Therefore, I will drop all records > 1000

In [126]:
numeric_df = numeric_df[numeric_df.num_private < 1000]

### 2.3.4 population

In [127]:
# checking the percentage of 0 values
zero_count_4 = numeric_df.population.value_counts()[0]
zero_count_4 / len(numeric_df.population)

0.35997979627914806

In [129]:
numeric_df.population.sort_values(ascending=False).head(60)

30537    30500
14308    15300
51398    11463
30231    10000
56742    10000
31363    10000
12784     9865
13725     9500
31473     9000
48725     9000
51919     9000
18798     8848
11426     8600
22369     8500
58781     8200
9021      8000
52947     8000
26985     8000
46922     8000
59096     8000
37826     7530
23830     7500
27653     7500
3853      7500
29905     7000
4905      7000
17894     6922
24644     6922
56580     6922
14949     6922
16370     6922
25445     6922
52178     6922
45154     6922
58595     6922
53332     6922
17185     6922
26849     6922
43178     6922
35855     6922
9681      6922
39052     6922
49731     6922
869       6922
5262      6922
45797     6922
59075     6922
30246     6922
48286     6922
38973     6922
10955     6922
50888     6922
8779      6922
6228      6922
24180     6922
38148     6922
24255     6854
17810     6800
25822     6500
30561     6500
Name: population, dtype: int64

Clearly 30500 is an outlier. I am dropping it.